In [9]:
import glob
import csv
import pandas
import pprint
import csv
import re

In [10]:
working_directory = '/Users/mf019/bioinformatics/blastDB/BbPlasmid'

In [15]:
filenames_with_dupes = glob.glob('files/*.fa')
filenames_without_dupes = glob.glob('files_nodupes/*.fa')
files_3 = glob.glob('files_3/*.fa')
pprint.pprint(len(filenames_with_dupes))
pprint.pprint(len(filenames_without_dupes))
pprint.pprint(len(files_3))

725
258
410


In [ ]:

plasmid_dict_dupes = {}
i = 0
with open('ncbiID_to_plasmid.tsv', 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
    for filename in filenames_with_dupes:
        for line in open(filename):
            if line.startswith(">"):
                header = line.strip(">").replace(",","")
                # regex madness to tidy up the plasmid IDs
                pattern = r'.*?([cl]?p\d+(?:-?_?\+?\d+(?:-?_?\+?\d+)?)?(?!\w))' # ok this is a bit tricky but this is what I need.
                matches = re.findall(pattern, header)
                print(header, 'matches: ',matches)
                if matches == []:
                    plasmidID = 'chromosome'
                else:
                    plasmidID = ' '.join(matches)
                print(plasmidID, '\n')
                header = header.split(' ')
                NCBI_ID = header[0]
                if header[3] == 'strain':
                    strainID = header[4]
                    #plasmidID = header[6]
                else:
                    if header[3] == header[5]: # some of these fields are wonky
                        strainID = header[3]
                        #plasmidID = header[6]
                    else:
                        strainID = header[3]
                        #plasmidID = header[5]

                if strainID == ('plasmid' or 'strain' or 'isolate' or 'linear' or 'circular'):
                    print("something is wrong with this line!!!!!!!")
                    print(header)
                    print("!!!!!")

                writer.writerow([NCBI_ID, strainID, plasmidID])
        if strainID not in plasmid_dict_dupes:
            plasmid_dict_dupes[strainID] = []
            #print("making new key")
        plasmid_dict_dupes[strainID].append((NCBI_ID, plasmidID))
        #print("appending")
        #print(i)
        i += 1



In [ ]:
print(len(plasmid_dict_dupes))
pprint.pprint(plasmid_dict_dupes)
#n plasmid_dict_dupes:
#    pprint.pprint(key)

In [ ]:
plasmid_dict_nodupes = {}
i = 0
with open('ncbiID_to_plasmid2.tsv', 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
    for filename in filenames_without_dupes:
        for line in open(filename):
            if line.startswith(">"):
                header = line.strip(">").replace(",","")
                # regex madness to tidy up the plasmid IDs
                pattern = r'.*?([clp]{1,2}\d+(?:-?_?\+?\d+(?:-?_?\+?\d+)?)?(?!\w))' # ok this is a bit tricky but this is what I need.
                matches = re.findall(pattern, header)
                print(matches)
                if matches is None:
                    plasmidID = 'chromosome'
                else:
                    plasmidID = ' '.join(matches)

                header = header.split(' ')
                NCBI_ID = header[0]
                print(header)
                if header[3] == 'strain':
                    strainID = header[4]
                    #plasmidID = header[6]
                else:
                    if header[3] == header[5]: # some of these fields are wonky
                        strainID = header[3]
                        #plasmidID = header[6]
                    else:
                        strainID = header[3]
                        #plasmidID = header[5]
                if strainID == 'circular':
                    print("something is wrong with this line!!!!!!!")
                    print(header)
                    print("!!!!!")
                #print("file: ",filename)
                #print("NCBI ID: ", header[0])
                #print("strain: ", strainID)
                #print("plasmid: ",plasmidID)
                #print("********")

                writer.writerow([NCBI_ID, strainID, plasmidID])
        if strainID not in plasmid_dict_nodupes:
            plasmid_dict_nodupes[strainID] = []
                #print("making new key")
        plasmid_dict_nodupes[strainID].append((NCBI_ID, plasmidID))
        #print("appending")
        #print(i)
        i += 1



In [ ]:
pprint.pprint(plasmid_dict_nodupes)

In [ ]:
print(len(plasmid_dict_dupes))
for key in plasmid_dict_dupes:
    pprint.pprint(key)

print(len(plasmid_dict_nodupes))
for key in plasmid_dict_nodupes:
    pprint.pprint(key)

In [ ]:
plasmids = {strain : [{ NCBI_ID : {plasmid : seq}}, plasmid_matrix]}
class plasmid:
    plasmids_present = {plasmid1: (yes/no), plasmid2: (yes/no), ...}

# ok so first I need to get all of the strains set aside in their own little group.
# then I need to get all of the plasmids present for each strain in their group
# then I need to identify which are present and then set those into the data object
# then use that to identify which strain is present based on construction of a new object given strain assignment.

# something like

for contig1:
    top_hit = hit1
    top_hit.score = hit1.score
    top_hit.ID = hit1.ID
    top_hit.length = hit1.length
    top_hit.percentIdent = hit1.percentIdent
    # or some iteration of this.
    # then take and parse out the top hit, or rather
    # look through all hits and then parse the ID given by blast to determine which plasmid it says it is.
    # then use the score, length, percent ident, etc to assign some score to the hit
    # then use all of the scores to assign probability and then use to determine plasmid assignment
    # then after that, use all of the plasmid assignment values to determine strain identification
    # would likely need a probability matrix for each strain given the plasmids present..........